# Begin of ADM 3

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import pandas as pd
import numpy as np

### 1.1. Get the list of places

In [ ]:
with open('Places_url.txt', 'x') as f:
    for i in tqdm(range(1,401)):             
        url = f'https://www.atlasobscura.com/places?page={i}&sort=likes_count'
        result = requests.get(url)
        soup = BeautifulSoup(result.text)
        matches = soup.find_all("a", {"class": "content-card content-card-place"})
        for match in matches:
            f.write(match['href'] + '\n')


### 1.2. Crawl places

In [ ]:
os.mkdir(f'Locations')
places_url = pd.read_csv('Places_url.txt', header=None)


for page in tqdm(range(0,400)):
    os.mkdir(f'Locations/page_{page + 1}')
    for loc in range(0,18):
        with open(f'Locations/page_{page + 1}/location_{(18 * page) + loc + 1}.html', 'x', encoding="utf-8") as f:
            url = 'https://www.atlasobscura.com' + places_url.iloc[(18 * page) + loc][0]
            result = requests.get(url)
            html = result.text
            f.write(html)
        

### 1.3 Parse downloaded pages

In [120]:
# example not NA
url = 'https://www.atlasobscura.com/places/highgate-cemetery'
index = requests.get(url)
soup = BeautifulSoup(index.text)

In [137]:
# NA presence
url = 'https://www.atlasobscura.com/places/gjain-thjorsardalur'
index = requests.get(url)
soup = BeautifulSoup(index.text)

In [138]:
from datetime import datetime
# put it on the 'import' cell ☺
# and please check BeautifulSoup or bs and put them in the say way
# THANKS!


# 7)

placeNearby = [match.text for match in soup.find_all('div', {'class': 'DDPageSiderailRecirc__item-title'})]
if placeNearby == []:
    placeNearby = ''

    
# 8)

match = soup.find_all("address", {"class":"DDPageSiderail__address"})
if match == []:
    placeAddress = ''
else:
    a = match[0].find('div').contents[0:5:2]
    placeAddress = " ".join(a)  


# 9)
# ask for int but are coordinate are float

match = soup.find_all("div", {"class":"DDPageSiderail__coordinates js-copy-coordinates"})
if match != []:
    match = match[0].get_text().split()
    placeAlt, placeLong = float(match[0][:-1]), float(match[1])
else: 
    placeAlt, placeLong = '', ''


# 10)

placeEditors = [match.text.strip() for match in soup.find_all("a", {'class' : "DDPContributorsList__contributor"})]
if placeEditors == []:
    placeEditors = ''
    

# 11)

date = soup.find_all("div", {"class":"DDPContributor__name"})
if date != []:
    date = date[0].text
    placePubDate = datetime.strptime(date, '%B %d, %Y')
else:
    placePubDate = ''


# 12)
    
a = soup.find_all('div', {"data-gtm-template" : "DDP Footer Recirc Lists"})
if a != []:
    placeRelatedLists = [match.text.strip() for match in a[0].find_all('h3')]
else: 
    placeRelatedLists = ''
    

# 13)

matches = soup.find_all("div", {'data-gtm-template': "DDP Footer Recirc Related"})
if matches == []:
    placeRelatedPlaces = ''
else:
    matches = matches[0].find_all('h3', {'class':"Card__heading --content-card-v2-title js-title-content"})
    placeRelatedPlaces = [match.text.strip() for match in matches]
    
    
    
# 14)

url = soup.find_all("link", {'rel':'canonical'})
if url != []:
    placeURL = url[0]['href']
else:
    placeURL = ''

## ce provamo